### PIPs and imports

In [1]:
!pip install transformers -q
!pip install sentencepiece -q
!pip install datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 93.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 22.8 MB/s eta 0:00:00


In [156]:
# evaluate might get an error when installing
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install -q evaluate
!pip install -q rouge_score
!pip install -q git+https://github.com/google-research/bleurt.git

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import re
from urllib.request import urlopen
from sklearn.model_selection import train_test_split

In [157]:
import datasets
from datasets import load_dataset, load_metric, Dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AutoTokenizer
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import BertTokenizerFast, BertTokenizer
from transformers import EncoderDecoderModel, EncoderDecoderConfig
import evaluate
import torch

In [133]:
n_context_sentences = 3

trainval_file = f'/content/drive/MyDrive/Courses/UCB/DS266 - NLP/final_project/posptproc_corpus_spacy_s{n_context_sentences}_train_val.csv'
test_file = f'/content/drive/MyDrive/Courses/UCB/DS266 - NLP/final_project/posptproc_corpus_spacy_s{n_context_sentences}_test.csv'
preds_file_out = f'/content/drive/MyDrive/Courses/UCB/DS266 - NLP/final_project/bert2bert_s{n_context_sentences}_test_preds.csv'
s_max_len = [65, 110, 150][n_context_sentences-1]

NUM_TRAIN_SAMPLES = 110_000 # Start with a small sample size.
NUM_VAL_SAMPLES = 45_000 # Start with a small sample size.
MAX_LOAD_AT_ONCE = 2_500
MAX_TOKEN_LENGTH = 128

MODEL_CKPT_FOLDER = '/content/drive/MyDrive/Courses/UCB/DS266 - NLP/final_project/checkpoints/'
MODEL_CKPT_FILE = MODEL_CKPT_FOLDER + 'xlnbase-finetuned'
TUNED_XLN_SAVED = '/content/drive/MyDrive/Courses/UCB/DS266 - NLP/final_project/saved_models/gpt2_finetuned'
PROMPT = 'generate next line: '
BATCH_SIZE = 16
SEED = 42

### Dataset

In [134]:
# read train
df_trainval = pd.read_csv(trainval_file)

# create train val split
n_rows = df_trainval.shape[0]
train_sample_size = int(n_rows * 0.88)
val_sample_size = n_rows - train_sample_size
n_rows, train_sample_size, val_sample_size, train_sample_size + val_sample_size

np.random.seed(SEED)
train_indices = np.random.choice(range(n_rows), train_sample_size, replace=False)
val_indices = list(set(range(n_rows)) - set(train_indices))
df_train = df_trainval.iloc[train_indices].reset_index(drop=True)
df_val = df_trainval.iloc[val_indices].reset_index(drop=True)
print(df_train.shape[0], df_val.shape[0], n_rows)

162536 22164 184700


In [137]:
hf_train_dset = Dataset.from_pandas(df_train)
hf_val_dset = Dataset.from_pandas(df_val)
hf_train_dset, hf_val_dset

(Dataset({
     features: ['variable', 'label'],
     num_rows: 162536
 }),
 Dataset({
     features: ['variable', 'label'],
     num_rows: 22164
 }))

In [8]:
# need the decoder to tokenize
bert_tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")

In [138]:
# pre process the data
def encoder_decoder_pre_proc(batch):
  inputs = bert_tokenizer(batch["variable"], padding="max_length", truncation=True, max_length=s_max_len)
  outputs = bert_tokenizer(batch["label"], padding="max_length", truncation=True, max_length=s_max_len)

  batch['input_ids'] = inputs.input_ids
  batch['attention_mask'] = inputs.attention_mask
  batch["label_ids"] = outputs.input_ids.copy()

  batch["label_ids"] = [[-100 if token == bert_tokenizer.pad_token_id else token for token in labels] for labels in batch["label_ids"]]

  return batch

In [139]:
# train
hf_train_dset_tkn = hf_train_dset.map(
    encoder_decoder_pre_proc,
    batched=True,
    batch_size=BATCH_SIZE,
    remove_columns=['variable', 'label']
)

# train_dset = train_dset.set_format(type="torch", columns=["input_ids", "attention_mask", "label_ids"])
hf_train_dset_tkn

Map:   0%|          | 0/162536 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'label_ids'],
    num_rows: 162536
})

In [140]:
hf_val_dset_tkn = hf_val_dset.map(
    encoder_decoder_pre_proc,
    batched=True,
    batch_size=BATCH_SIZE,
    remove_columns=['variable', 'label']
)

# val_dset = val_dset.set_format(type="torch", columns=["input_ids", "attention_mask", "label_ids"])
hf_val_dset_tkn

Map:   0%|          | 0/22164 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'label_ids'],
    num_rows: 22164
})

In [141]:
# create model with BERT as encoder and decoder
bert2bert = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-cased", "bert-base-cased")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationshi

In [142]:
# add special tokens for decoder that BERT doesn't have
bert2bert.config.decoder_start_token_id = bert_tokenizer.cls_token_id
bert2bert.config.eos_token_id = bert_tokenizer.sep_token_id
bert2bert.config.pad_token_id = bert_tokenizer.pad_token_id
bert2bert.config.vocab_size = bert2bert.config.encoder.vocab_size

In [143]:
training_args = Seq2SeqTrainingArguments(
    output_dir=MODEL_CKPT_FOLDER,
    save_total_limit=1,
    num_train_epochs=3,
    load_best_model_at_end=True,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    save_strategy='epoch',
    evaluation_strategy='epoch'
)

In [144]:
trainer = Seq2SeqTrainer(
    model=bert2bert,
    tokenizer=bert_tokenizer,
    args=training_args,
    train_dataset=hf_train_dset_tkn,
    eval_dataset=hf_val_dset_tkn,
)

trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.9/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pas

Epoch,Training Loss,Validation Loss
1,3.403500,3.307594
2,3.105400,3.155852
3,2.872700,3.124666


/usr/local/lib/python3.9/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/usr/local/lib/python3.9/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no ne

TrainOutput(global_step=30477, training_loss=3.2306730095966323, metrics={'train_runtime': 9846.7722, 'train_samples_per_second': 49.52, 'train_steps_per_second': 3.095, 'total_flos': 8.76336448778208e+16, 'train_loss': 3.2306730095966323, 'epoch': 3.0})

In [145]:
trainer.save_model(f'/content/drive/MyDrive/Courses/UCB/DS266 - NLP/final_project/saved_models/bert2bert_s{n_context_sentences}_e3_mxlen{s_max_len}')

### Generate text

In [148]:
# read train
df_test = pd.read_csv(test_file)
hf_test = Dataset.from_pandas(df_test)
hf_test

Dataset({
    features: ['variable', 'label'],
    num_rows: 20523
})

In [149]:
print(n_context_sentences, s_max_len)
bert2bert_loaded = EncoderDecoderModel.from_pretrained(f"/content/drive/MyDrive/Courses/UCB/DS266 - NLP/final_project/saved_models/bert2bert_s{n_context_sentences}_e3_mxlen{s_max_len}").to("cuda")

3 150


In [151]:
def generate_summary(batch):
  inputs = bert_tokenizer(batch['variable'], padding="max_length", truncation=True, max_length=s_max_len, return_tensors='pt')
  input_ids = inputs.input_ids.to("cuda")
  # attention_mask = inputs.attention_mask.to("cuda")

  outputs = bert2bert_loaded.generate(
      input_ids, 
      # attention_mask=attention_mask,
      max_new_tokens=50,
      num_beams=4,
      no_repeat_ngram_size=2,
      early_stopping=True,
      num_return_sequences=1)
  
  output_str = bert_tokenizer.batch_decode(outputs, skip_special_tokens=True)

  batch["pred_label"] = output_str

  return batch

In [152]:
hf_test_preds = hf_test.map(generate_summary, batched=True, batch_size=BATCH_SIZE)
hf_test_preds

Map:   0%|          | 0/20523 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


Dataset({
    features: ['variable', 'label', 'pred_label'],
    num_rows: 20523
})

In [155]:
df_test_preds = hf_test_preds.to_pandas()
df_test_preds

,variable,label,pred_label
0,Then his hare came and offered to go and bring...,"The King said, “It is not possible, the gates ...","Then the King ’ s son said to him, “ If thou w..."
1,Will you have hash or fishballs for breakfast?...,If she had known what birthday gift was coming...,""" I don't know what to do, "" she said."
2,"He almost choked with envy, and from the momen...",His own plants were neglected; it was useless ...,"Van Baerle, in spite of his disinterestedness,..."
3,"We will put another floor in our nest, right o...","They went to work once more, and it was not so...","He was a very good fellow, and he was very kin..."
4,Dat's what comes of havin' a cat wit' one blue...,Smith took advantage of the pause to leave the...,""" I don't know, "" he said."
...,...,...,...
20518,To shoot now would be to attract attention. Th...,"No, nothing could happen till they reached ope...",It was impossible for him to do so.
20519,Then it had occurred to him to work upon the s...,"“Not a thought,” replied Dr. Livesey cheerily.","“ No, sir, ” said Silver."
20520,Camellias were all the fashion. Everyone inqui...,I don’t know whether there was anything I mean...,"He was very fond of her, and loved her very much."
20521,THE CHILDREN ARE CARRIED OFF The pirate attack...,The white men have in the meantime made a rude...,It was the first time in his life that he had ...


### Evaluate

In [176]:
bleu = evaluate.load('bleu')
rouge = evaluate.load('rouge')
bleurt = evaluate.load('bleurt', 'bleurt-large-512', download_config=datasets.download.DownloadConfig())

In [177]:
def get_model_metrics(preds, references):
  refs = [[s] for s in references]
  bleu_scores = bleu.compute(predictions=preds, references=refs)
  bleu_score = {'blue' : bleu_scores['bleu']}

  rouge_score = rouge.compute(predictions=preds, references=refs)

  bleurt_scores = bleurt.compute(predictions=preds, references=references)
  bleurt_score = {'bleurt_score' : np.average(bleurt_scores['scores'])}

  return pd.DataFrame(bleu_score | rouge_score | bleurt_score, index=['scores'])

In [178]:
dfs = []
for nsent in [1, 2, 3]:
  test_file = f'/content/drive/MyDrive/Courses/UCB/DS266 - NLP/final_project/bert2bert_s{nsent}_test_preds.csv'
  df_tp = pd.read_csv(test_file)
  df = get_model_metrics(df_tp['pred_label'].tolist(), df_tp['label'].tolist())
  df['n_context_sents'] = f'S{nsent}'
  dfs.append(df)
pd.concat(dfs).reset_index(drop=True)

,blue,rouge1,rouge2,rougeL,rougeLsum,bleurt_score,n_context_sents
0,0.010602,0.119237,0.014406,0.101568,0.101565,-1.109143,S1
1,0.011956,0.126579,0.017824,0.108175,0.108119,-1.098777,S2
2,0.012474,0.130512,0.019387,0.111407,0.111362,-1.079424,S3


In [123]:
# score by example
def calculate_bleu(x):
  bleu_score = bleu.compute(predictions=[x['pred_label']], references=[[x['label']]])
  return bleu_score['bleu']

df_test_preds['bleu'] = df_test_preds.apply(calculate_bleu, axis=1)

In [124]:
df_test_preds['bleu'].describe()

count    20571.000000
mean         0.003666
std          0.034486
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: bleu, dtype: float64